In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:25pt;}
div.text_cell_render.rendered_html{font-size:18pt;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:21pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

<b><font color="red" size="6">ch11. 데이터프레임과 시리즈(Pandas)</font></b>
# 9절. 데이터 그룹화 및 집계

In [5]:
from statsmodels.api import datasets
iris = datasets.get_rdataset('iris').data
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
25,5.0,3.0,1.6,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica
125,7.2,3.2,6.0,1.8,virginica


## 9.1 Goupby
cf. pivot_table

In [16]:
# 1) 그룹화 방법1
#      df.groupby(by=[그룹핑할열])[함수적용해서 출력할 열들].기초통계함수()
#             [함수적용해서 출력할 열들] 생략시 그룹핑할열을 제외한 모든 열 적용  
iris_grouped = iris.groupby(iris.Species)
iris_grouped = iris.groupby(['Species'])
iris_grouped = iris.groupby('Species')
for group, data in iris_grouped:
    print(group)
    display(data.sample(2).sort_index())

setosa


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
11,4.8,3.4,1.6,0.2,setosa


versicolor


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
81,5.5,2.4,3.7,1.0,versicolor
98,5.1,2.5,3.0,1.1,versicolor


virginica


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
134,6.1,2.6,5.6,1.4,virginica
148,6.2,3.4,5.4,2.3,virginica


In [32]:
display(iris.groupby(by='Species').mean()[['Petal.Length','Petal.Width']])
display(iris.groupby(by='Species')[['Petal.Length','Petal.Width']].mean())

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [29]:
iris_grouped_sum = iris.groupby('Species').median()
# iris_grouped_sum.reset_index()
iris_grouped_sum

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3
virginica,6.5,3.0,5.55,2.0


In [28]:
iris_grouped_sum.loc[['setosa','versicolor']]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3


In [24]:
iris_grouped_sum2 = iris.groupby('Species', as_index=False).sum()
iris_grouped_sum2

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3
2,virginica,329.4,148.7,277.6,101.3


In [31]:
iris_grouped_sum2[(iris_grouped_sum2.Species=='setosa') | 
                  (iris_grouped_sum2.Species=='versicolor')]

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3


In [ ]:
#      df.groupby(by=[그룹핑할열])[함수적용해서 출력할 열들].함수()
# 2) 그룹화 방법2
#     df.pivot_table(index=[그룹핑할열],
    #                values=[함수적용해서 출력할 열들],
    #                aggfunc=함수(기본값:mean))

In [37]:
iris.groupby('Species')['Petal.Length', 'Petal.Width'].mean()
iris.pivot_table(index='Species',
                values=['Petal.Length', 'Petal.Width'],
                aggfunc='mean')

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [39]:
iris.groupby('Species').count() # 결측치를 제외한 값의 갯수
iris.pivot_table(index='Species',
                #values=['Sepal.Length','Sepal.Width','Petal.Length','Petal.Width'],
                 aggfunc='count')

,Petal.Length,Petal.Width,Sepal.Length,Sepal.Width
Species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


In [41]:
iris.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


### 다중열로 그룹화

In [42]:
# iris로 다중열로 그룹화하기 위해 데이터 컬럼 추가(num 필드)
iris.shape

(150, 5)

In [47]:
import numpy as np
np.ravel([[i]*25 for i in range(6)])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [50]:
iris['num'] = np.ravel([[i]*25 for i in range(6)])
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,num
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,1
50,7.0,3.2,4.7,1.4,versicolor,2
75,6.6,3.0,4.4,1.4,versicolor,3
100,6.3,3.3,6.0,2.5,virginica,4
125,7.2,3.2,6.0,1.8,virginica,5


In [ ]:
# 각열의 임의의 데이터에 결측치를 할당
# iris.iloc[0, 0] = np.nan

In [55]:
import random
print(random.sample(range(149), 3))
iris.iloc[random.sample(range(149), 3), 0]

35    5.0
93    5.0
96    5.7
Name: Sepal.Length, dtype: float64